### **■ 유통 판매량 예측 및 재고 최적화**
# **단계 1: 데이터 탐색**

<img src = "https://github.com/Jangrae/img/blob/master/store.png?raw=true" width=800, align="left"/>

# **⏰ 수행 과제**

다음과 같은 과정으로 프로젝트를 진행합니다.

#### **1. 환경 설정**
- 이후 진행에 필요한 환경 설정을 수행합니다.


#### **2. 데이터 탐색**
- 대상 데이터에 대해 충분히 탐색합니다.
- 특히 시계열 패턴을 찾기 위한 다양한 시각화를 수행합니다.
- 탐색 결과를 기반으로 이후에 새로운 변수를 추가하게 됩니다.


#### **📢 예측해야 할 핵심 상품 역할 분담**
- 핵심 상품별로 데이터를 탐색하고 가설을 수립해야 합니다.
- 2인당 핵심 상품 하나씩 분담하여 과제를 수행합니다.
- 팀 미팅을 통해 진행 사항을 공유하고 내용을 취합해 정리합니다.


#### **📢 대상 상품(핵심 상품)**

| Product_ID|Product_Code|SubCategory|Category|LeadTime|Price|
|----|----|----|----|----|----|
|3|DB001|Beverage|Drink|2|8|
|12|GA001|Milk|Food|2|6|
|42|FM001|Agricultural products|Grocery|2|5|

# **1. 환경 설정**

- 이후 진행에 필요한 환경 설정을 수행합니다.

## **(1) 경로 설정**

- 프로젝트 수행 환경에 맞게 파일 경로를 설정합니다.

### **1) 로컬 수행(Anaconda)**
- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [ ]:
# 기본 경로
path = ''

### **2) 구글 콜랩 수행**

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [ ]:
# 구글 드라이브 연결, 패스 지정
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/project/'

## **(2) 라이브러리 불러오기**

- 이후 사용할 기본 라이브러리를 불러옵니다.

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

## **(3) 데이터 불러오기**

- 이후 분석 대상이 되는 파일을 불러오고 기본 정보를 확인합니다.

### **1) 데이터 불러오기**

- 분석 대상인 학습용 데이터를 불러옵니다.
- 데이터프레임 이름은 다음과 같이 통일합니다.
    - sales: 판매 정보
    - orders: 고객 방문수
    - oil_price: 휘발유 가격
    - stores: 매장 정보
    - products: 상품 정보
- 날짜 데이터를 갖는 Date 변수는 이후 편의를 위해 datetime 형으로 변환합니다.

In [ ]:
# 데이터 불러오기
sales = pd.read_csv(path + 'sales_train.csv')
orders = pd.read_csv(path + 'orders_train.csv')
oil_price = pd.read_csv(path + 'oil_price_train.csv')
stores = pd.read_csv(path + 'stores.csv')
products = pd.read_csv(path + 'products.csv')

In [ ]:
# datetime 형으로 변환
sales['Date'] = pd.to_datetime(sales['Date'] )
oil_price['Date'] = pd.to_datetime(oil_price['Date'] )
orders['Date'] = pd.to_datetime(orders['Date'] )

### **2) 기본 정보 확인**

- 각 데이터의 기본 정보를 확인합니다.
- Date 변수가 있는 데이터의 날짜 범위(최솟값, 최댓값)를 확인합니다.

In [ ]:
# sales
display(sales.head())
print(sales['Date'].min(), '~', sales['Date'].max())

In [ ]:
# orders
display(orders.head())
print(orders['Date'].min(), '~', orders['Date'].max())

In [ ]:
# oil_price
display(oil_price.head())
print(oil_price['Date'].min(), '~', oil_price['Date'].max())

In [ ]:
# products
products.head()

In [ ]:
# stores
stores.head()

# **2. 데이터 탐색**

- 대상 데이터에 대해 충분히 탐색합니다.
- 특히 시계열 패턴을 찾기 위한 다양한 시각화를 수행합니다.
- 탐색 결과를 기반으로 이후에 새로운 변수를 추가하게 됩니다.

## **(1) 기본 데이터 탐색**

- 개별 데이터에 대한 충분한 탐색을 수행합니다.
- 다음과 같은 탐색이 포함될 수 있습니다.
    - 각 데이터 결측치 존재 여부 확인
    - 각 데이터에 포함된 범줏값 비율 확인
    - 각 데이터에 포함된 연속값 범위와 분포 확인
    - 기타 등등...

### **1) 결측치 확인**

- 데이터로 결측치 존재 여부를 확인합니다.

In [ ]:
# 결측치 확인
files = ['sales', 'products', 'orders', 'stores', 'oil_price']
for f in files:
    print('=' * 20)
    print(f)
    print('-' * 20)
    print(eval(f'{f}.isna().sum()'))

### **2) 범줏값 비율 확인**

- 중요 범줏값의 비율을 확인합니다.

In [ ]:
# sales
sales['Product_ID'].value_counts()

In [ ]:
# products
print(products['Category'].value_counts())
print('-' * 30)
print(products['SubCategory'].value_counts())
print('-' * 30)
print(products['LeadTime'].value_counts())

## **(2) 판매량, 고객 방문수**

- 시각화를 통해 상품별 판매량, 고객 방문수 등을 월별로 살펴봅니다.
- 대상 매장(44), 대상 상품(3, 12, 42)에 대해서 살펴봅니다.
- 다음과 같은 시각화가 포함될 수 있습니다.
    - 3년간(2014년~2016년)상품별 월별 판매량
    - 2016년 상품별 월별 판매량
    - 3년간(2014년~2016년) 월별 고객 방문수
    - 2016년 월별 고객 방문수
    - 기타 등등...

### **1) 상품별 월별 판매 수량**

- 주요 상품의 월별 판매량을 확인합니다.

In [ ]:
# 함수 만들기
def monthly_sales_qty(Product_ID, sdate, edate):
    temp = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID) & (sales['Date'].between(sdate, edate))]
    temp['Month'] = temp['Date'].dt.month
    monthly = temp.groupby(by='Month', as_index=False)[['Qty']].sum()

    plt.figure(figsize=(12, 3))
    plt.bar(x=monthly['Month'], height=monthly['Qty'] / 1000)
    plt.title('Monthly Sales Qty', size=15, pad=30)
    plt.suptitle(f'Product: {Product_ID}, Period: {sdate} ~ {edate}', size=9)
    plt.xlabel('Month')
    plt.ylabel('Qty(Thousand)')
    plt.xticks(range(1, 13), range(1, 13))
    plt.show()

- 3년간(2014년 ~ 2016년) 월별 판매량을 확인합니다.

In [ ]:
# 상품별 2014년~2016년 월별 판매량
main_product = [3, 12, 42]
for p in main_product:
    monthly_sales_qty(p, '2014-01-01', '2016-12-31')

- 1년간(2016년) 월별 판매량을 확인합니다.

In [ ]:
# 상품별 2016년 월별 판매량
main_product = [3, 12, 42]
for p in main_product:
    monthly_sales_qty(p, '2016-01-01', '2016-12-31')

### **2) 월별 고객 방문수**

- 월별 고객 방문수를 확인합니다.

In [ ]:
orders.info()

In [ ]:
# 함수 만들기
def monthly_customer_visits(sdate, edate):
    temp = orders.loc[(orders['Store_ID']==44) & (orders['Date'].between(sdate, edate))]
    temp['Month'] = temp['Date'].dt.month
    monthly = temp.groupby(by='Month', as_index=False)[['CustomerCount']].sum()

    plt.figure(figsize=(12, 3))
    plt.bar(x=monthly['Month'], height=monthly['CustomerCount'] / 1000)
    plt.title('Monthly Customer Visits', size=15, pad=30)
    plt.suptitle(f'Period: {sdate} ~ {edate}', size=9)
    plt.xlabel('Month')
    plt.ylabel('Visits(Thousand)')
    plt.xticks(range(1, 13), range(1, 13))
    plt.show()

- 3년간(2014년 ~ 2016년) 월별 고객 방문수를 확인합니다.

In [ ]:
# 2014년~2016년 월별 고객 방문수
monthly_customer_visits('2014-01-01', '2016-12-31')

- 1년간(2016년) 월별 고객 방문수를 확인합니다.

In [ ]:
# 2016년 월별 고객 방문수
monthly_customer_visits('2016-01-01', '2016-12-31')

## **(3) 시계열 패턴 찾아보기 #1**

- 판매량을 선그래프로 시각화해 시계열 패턴을 찾아 봅니다.
- 대상 기간을 변경해 가면서 패턴을 확인해 보길 권고합니다.
- 다음과 같은 시각화가 포함될 수 있습니다.
    - 3년간(2014년~2016년)상품별 판매량 추이
    - 2016년 상품별 판매량 추이
    - 대상 상품과 같은 카테고리의 상품별 판매량 추이
    - 휘발류 가격과 상품 판매량 추이 비교
    - 방문 고객수와 상품 판매량 추이 비교
    - 기타 등등...

### **1) 상품별 판매량 추이**

- 대상 상품의 판매량 추이를 선그래프로 시각화해 확인합니다.

In [ ]:
# 함수 만들기
def product_sales_trend(Product_ID, sdate, edate):
    temp = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID) & (sales['Date'].between(sdate, edate))]
    # 시각화
    plt.figure(figsize=(12, 3))
    sns.lineplot(x='Date', y = 'Qty', data = temp)
    plt.title(f'Product Sales Trend', size=15, pad=30)
    plt.suptitle(f'Product: {Product_ID}, Period: {sdate} ~ {edate}', size=9)
    plt.xlabel('Date')
    plt.ylabel('Qty')
    plt.show()

In [ ]:
# 3번 상품 판매량 추이
product_sales_trend(3, '2014-01-01', '2016-12-31')
product_sales_trend(3, '2016-01-01', '2016-12-31')

In [ ]:
# 12번 상품 판매량 추이
product_sales_trend(12, '2014-01-01', '2016-12-31')
product_sales_trend(12, '2016-01-01', '2016-12-31')

In [ ]:
# 42번 상품 판매량 추이
product_sales_trend(42, '2014-01-01', '2016-12-31')
product_sales_trend(42, '2016-01-01', '2016-12-31')

### **2) 대상 상품과 같은 카테고리의 상품별 판매량 추이**

- 대상 상품과 같은 카테고리에 포함된 상품 판매량을 집계해 판매량 추이를 선그래프로 시각화해 확인합니다.

In [ ]:
# 함수 만들기
def category_product_sales_trend(Product_ID, sdate, edate, each=0):
    Category = products.loc[products['Product_ID'].isin([Product_ID]), 'Category'].to_list()
    Product_IDs = products.loc[products['Category'].isin(Category), 'Product_ID'].to_list()

    temp1 = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID'].isin(Product_IDs)) & (sales['Date'].between(sdate, edate))]
    temp1 = temp1.groupby(by='Date', as_index = False)[['Qty']].sum()

    temp2 = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID) & (sales['Date'].between(sdate, edate))]

    plt.figure(figsize = (12, 3))
    plt.plot(temp1['Date'], temp1['Qty'], label='Same Category Product')
    plt.plot(temp2['Date'], temp2['Qty'], label=f'{Product_ID} Product')
    plt.title('Same Category Product Sales Trend', size=15, pad=30)
    plt.suptitle(f'Period: {sdate} ~ {edate}', size=9)
    plt.legend()
    plt.xlabel('Date')
    plt.ylabel('Qty')
    plt.show()

In [ ]:
# 3번 상품 동일 카테고리 상품 판매량 추이
category_product_sales_trend(3, '2016-01-01', '2016-12-31')

In [ ]:
# 12번 상품 동일 카테고리 상품 판매량 추이
category_product_sales_trend(12, '2016-01-01', '2016-12-31')

In [ ]:
# 42번 상품 동일 카테고리 상품 판매량 추이
category_product_sales_trend(42, '2016-01-01', '2016-12-31')

In [ ]:
# 42번 상품 판매량 추이
product_sales_trend(42, '2016-01-01', '2016-12-31')

### **3) 휘발유 가격과 상품 판매량 추이 비교**

- 휘발유 가격 추이와 대상 상품 판매량 추이를 선그래프로 시각화해 비교합니다.

In [ ]:
# 함수 만들기
def product_oil_trend(Product_ID, sdate, edate):
    temp1 = oil_price.loc[oil_price['Date'].between(sdate, edate)]
    temp2 = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID) & (sales['Date'].between(sdate, edate))]

    plt.figure(figsize=(12, 5))
    plt.suptitle('Product Sales Trend With Oil Price', size=15)
    plt.subplot(2, 1, 1)
    plt.plot(temp1['Date'], temp1['WTI_Price'])
    plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.title(f'Product: {Product_ID}, Period: {sdate} ~ {edate}', size=9)
    plt.ylabel('Oil Price')
    plt.subplot(2, 1, 2)
    plt.plot(temp2['Date'], temp2['Qty'])
    plt.xlabel('Date')
    plt.ylabel('Qty')
    plt.tight_layout()
    plt.show()

In [ ]:
# 3번 상품
product_oil_trend(3, '2016-01-01', '2016-12-31')

In [ ]:
# 12번 상품
product_oil_trend(12, '2016-01-01', '2016-12-31')

In [ ]:
# 42번 상품
product_oil_trend(42, '2016-01-01', '2016-12-31')

### **4) 14일 동안 휘발유 가격 평균과 상품 판매량 추이 비교**

- 일별 휘발유 가격 변동보다는 최근 14일 동안의 휘발유 가격 평균값의 추이와 비교합니다.

In [ ]:
# 함수 만들기
def product_oil_trend(Product_ID, sdate, edate):
    temp1 = oil_price.loc[oil_price['Date'].between(sdate, edate)]
    temp1['WTI_Price'] =  temp1['WTI_Price'].rolling(14, min_periods=1).mean()
    temp2 = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID) & (sales['Date'].between(sdate, edate))]

    plt.figure(figsize=(12, 5))
    plt.suptitle('Product Sales Trend With Oil Price', size=15)
    plt.subplot(2, 1, 1)
    plt.plot(temp1['Date'], temp1['WTI_Price'])
    plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.title(f'Product: {Product_ID}, Period: {sdate} ~ {edate}', size=9)
    plt.ylabel('Oil Price')
    plt.subplot(2, 1, 2)
    plt.plot(temp2['Date'], temp2['Qty'])
    plt.xlabel('Date')
    plt.ylabel('Qty')
    plt.tight_layout()
    plt.show()

In [ ]:
# 3번 상품
product_oil_trend(3, '2016-01-01', '2016-12-31')

In [ ]:
# 12번 상품
product_oil_trend(12, '2016-01-01', '2016-12-31')

In [ ]:
# 42번 상품
product_oil_trend(42, '2016-01-01', '2016-12-31')

### **5) 고객 방문수와 상품 판매량 추이 비교**

- 고객 방문수 추이와 대상 상품 판매량 추이를 선그래프로 시각화해 비교합니다.

In [ ]:
# 함수 만들기
def product_customer_trend(Product_ID, sdate, edate):
    temp1 = orders.loc[(orders['Store_ID']== 44) & orders['Date'].between(sdate, edate)]
    temp2 = sales.loc[(sales['Store_ID'] == 44) & (sales['Product_ID'] == Product_ID) & (sales['Date'].between(sdate, edate))]

    plt.figure(figsize=(12, 5))
    plt.suptitle('Product Sales Trend With Customer Visits', size=15)
    plt.subplot(2, 1, 1)
    plt.plot(temp1['Date'], temp1['CustomerCount'])
    plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.title(f'Product: {Product_ID}, Period: {sdate} ~ {edate}', size=9)
    plt.ylabel('Visits')
    plt.subplot(2, 1, 2)
    plt.plot(temp2['Date'], temp2['Qty'])
    plt.xlabel('Date')
    plt.ylabel('Qty')
    plt.tight_layout()
    plt.show()

In [ ]:
# 3번 상품
product_customer_trend(3, '2016-10-01', '2016-12-31')

In [ ]:
# 12번 상품
product_customer_trend(12, '2016-10-01', '2016-12-31')

In [ ]:
# 42번 상품
product_customer_trend(42, '2016-10-01', '2016-12-01')

## **(4) 시계열 패턴 찾아보기 #2**

- 상품별 판매량의 변화량을 비교합니다.
- **df['변수'].diff()** 형태로 변수에 대한 변화량을 확인할 수 있습니다.
- 변화량을 선그래프로 시각화하거나, 변화량의 분포를 히스토그램으로 시각화할 수 있습니다.

### **1) 상품별 변화량 비교**

In [ ]:
# 함수 만들기
def product_sales_diff_trend(Product_ID, sdate, edate):
    temp = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID) & (sales['Date'].between(sdate, edate))]
    temp['Qty_diff'] = temp['Qty'].diff()

    plt.figure(figsize=(12, 4))
    plt.suptitle('Product Sales Difference Trend', size=15)
    plt.subplot(1,2,1)
    plt.plot(temp['Date'], temp['Qty_diff'])
    plt.axhline(0, color='k', linewidth=0.3)
    plt.title(f'Product: {Product_ID}, Period: {sdate} ~ {edate}', size=9)
    plt.xlabel('Date')
    plt.subplot(1,2,2)
    plt.hist(x=temp['Qty_diff'], bins=20, alpha=0.7, ec='k')
    plt.title(f'Product: {Product_ID}, Period: {sdate} ~ {edate}', size=9)
    plt.xlabel('Difference')
    plt.tight_layout()
    plt.show()

In [ ]:
# 3번 상품
product_sales_diff_trend(3, '2016-01-01', '2016-12-31')

In [ ]:
# 12번 상품
product_sales_diff_trend(12, '2016-01-01', '2016-12-31')

In [ ]:
# 42번 상품
product_sales_diff_trend(42, '2016-01-01', '2016-12-31')

### **2) 요일별 변화량 비교**

- 요일별 판매량과 변화량을 막대그래프로 시각화해 비교합니다.
- 요일은 **df['Weekday'] = df['Date'].dt.weekday** 형태로 얻을 수 있습니다.
- Seaborn의 **sns.barplot()** 을 사용하면 자동 집계되어 시각화되니 편리합니다.

In [ ]:
def weekly_sales_diff_trend(Product_ID, sdate, edate):
    temp = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID) & (sales['Date'].between(sdate, edate))]
    temp['Qty_diff'] = temp['Qty'].diff()
    temp['Weekday'] = temp['Date'].dt.weekday

    plt.figure(figsize=(12, 4))
    plt.suptitle('Weekly Sales Difference Trend', size=15)
    plt.subplot(1, 2, 1)
    sns.barplot(x=temp['Weekday'], y=temp['Qty'])
    plt.title(f'Product: {Product_ID}, Period: {sdate} ~ {edate}', size=9)
    plt.xlabel('Weekday')
    plt.subplot(1, 2, 2)
    sns.barplot(x=temp['Weekday'], y=temp['Qty_diff'])
    plt.axhline(0, color='k', linewidth=0.3)
    plt.title(f'Product: {Product_ID}, Period: {sdate} ~ {edate}', size=9)
    plt.xlabel('Weekday')
    plt.tight_layout()
    plt.show()

In [ ]:
# 3번 상품
weekly_sales_diff_trend(3, '2016-01-01', '2016-12-31')

In [ ]:
# 12번 상품
weekly_sales_diff_trend(12, '2016-01-01', '2016-12-31')

In [ ]:
# 24번 상품
weekly_sales_diff_trend(24, '2016-01-01', '2016-12-31')

### **3) 시계열 데이터 분해**

- 다음과 같은 형태로 시계열 데이터를 분해할 수 있습니다.

~~~
decomp = sm.tsa.seasonal_decompose(temp['변수'], model='additive', period=7
~~~

- 분해 결과를 decomp 변수에 저장했으면 다음과 같이 데이터프레임을 만들어 시각화 할 수 있습니다.

~~~
result = pd.DataFrame({'observed':decomp.observed,
                       'trend':decomp.trend,
                       'seasonal':decomp.seasonal,
                       'residual':decomp.resid})
~~~

In [ ]:
# 함수 만들기
def decomp_plot(Product_ID, sdate, edate, period):
    # 시계열 데이터 분해
    temp = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID) & (sales['Date'].between(sdate, edate))]
    temp.reset_index(drop=True, inplace=True)
    decomp = sm.tsa.seasonal_decompose(temp['Qty'], model='additive', period=period)

    # 시계열 분해 결과를 받아서 데이터프레임으로 저장
    result = pd.DataFrame({'observed':decomp.observed,
                           'trend':decomp.trend,
                           'seasonal':decomp.seasonal,
                           'residual':decomp.resid})

    # 4개의 그래프로 나눠서 그리기
    plt.figure(figsize=(12, 8))
    # observed
    plt.subplot(4, 1, 1)
    plt.plot(result['observed'])
    plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.title('Time Series Decomposition', size=15)
    plt.ylabel('observed')
    # trend
    plt.subplot(4, 1, 2)
    plt.plot(result['trend'])
    plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.ylabel('trend')
    # seasonal
    plt.subplot(4, 1, 3)
    plt.plot(result['seasonal'])
    plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.ylabel('seasonal')
    # residual
    plt.subplot(4,1,4)
    plt.plot(result['residual'])
    plt.ylabel('residual')
    plt.tight_layout()
    plt.show()
    return result

In [ ]:
# 3번 상품
result = decomp_plot(3, '2016-01-01', '2016-12-31', 7)

In [ ]:
# 12번 상품
result = decomp_plot(12, '2016-01-01', '2016-12-31', 7)

In [ ]:
# 42번 상품
result = decomp_plot(42, '2016-01-01', '2016-12-31', 7)

### **4) 자기상관 분석**

- 자기상관 분석을 통해 이전 시점의 값과 어떤 관계를 가지는지를 확인합니다.

In [ ]:
# 모듈 불러오기
from statsmodels.graphics.tsaplots import plot_acf

# 함수 만들기
def auto_correlation(Product_ID, sdate, edate):
    temp = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID) & (sales['Date'].between(sdate, edate)), ['Qty']]
    fig, ax = plt.subplots(figsize=(12, 3))
    plot_acf(temp['Qty'], lags=50, alpha=0.05, ax=ax)
    plt.show()

In [ ]:
# 3번 상품
auto_correlation(3, '2016-01-01', '2016-12-31')

In [ ]:
# 12번 상품
auto_correlation(12, '2016-01-01', '2016-12-31')

In [ ]:
# 42번 상품
auto_correlation(42, '2016-01-01', '2016-12-31')

## **(5) 패턴 정리하기**

* 다양한 탐색을 통해 찾은 시계열 패턴을 적어보세요.